<a href="https://colab.research.google.com/github/VolodymyrKokhan/AB-test-project/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Опис проєкту:**

**Мета проекту** - *проаналізувати результати A/B-тестування за допомогою статистичних методів в Python та створити візуалізацію, що демонструє ключові конверсійні метрики.*

[Link to Tableau dashboard](https://public.tableau.com/app/profile/volodymyr.kokhan/viz/ABtest_17588319684430/Calculators/)

[Посилання на датафрейм з результатом](https://drive.google.com/file/d/1CxsJFb9j22AVF9PLkKbSqBpmGBLcxuRF/view?usp=sharing)

## **Імпорт необхідних біліотек**

In [ ]:
from google.colab import auth
from google.colab import drive
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

## **Підключення датасету**

In [ ]:
drive.mount("/content/drive")
ab_data = pd.read_csv("/content/drive/MyDrive/Mate/Portfolio/bq-results-20250925-135814-1758810063656.csv")
ab_data.info()
ab_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-03,New Zealand,tablet,Oceania,Organic Search,2,1,new account,1
1,2020-11-04,Tunisia,mobile,Africa,Organic Search,2,1,new account,1
2,2020-11-09,Jamaica,mobile,Americas,Organic Search,2,1,new account,1
3,2020-11-10,Puerto Rico,desktop,Americas,Paid Search,2,1,new account,1
4,2020-11-10,Iraq,mobile,Asia,Social Search,2,2,new account,1


**Виведемо список назв подій для подальшого їх використання в метриках**

In [ ]:
event_names = ab_data['event_name'].unique().tolist()
event_names

['new account',
 'session with orders',
 'session',
 'first_visit',
 'session_start',
 'user_engagement',
 'add_payment_info',
 'page_view',
 'view_promotion',
 'scroll',
 'view_search_results',
 'view_item',
 'add_to_cart',
 'add_shipping_info',
 'select_item',
 'select_promotion',
 'begin_checkout',
 'click',
 'view_item_list']

## **Створення функції, яка розраховує метрики АВ тесту**

In [ ]:
def calculate_ab_metrics(df: pd.DataFrame, numerator_event: str, denominator_event: str, alpha: float = 0.05) -> pd.DataFrame:
    # Розраховує метрики A/B-тестування (конверсію та статистичну значущість) для заданих подій.
    # Аргументи:
    # df (pd.DataFrame): Датафрейм з колонками 'test', 'test_group', 'event_name', 'value'.
    # numerator_event (str): Подія-чисельник (наприклад "add_payment_info")
    # denominator_event (str): Подія-знаменник (наприклад "session")
    # alpha (float): рівнь значущості, за замовчування 0.05
    # Повертає: Таблицю з результатами A/B-тесту.

    # Фільтруємо події
    numerator_data = df[df['event_name'] == numerator_event]
    denominator_data = df[df['event_name'] == denominator_event]

    # Групуємо та сумуємо події
    numerator_counts = numerator_data.groupby(['test', 'test_group'])['value'].sum().rename('numerator_count').reset_index()
    denominator_counts = denominator_data.groupby(['test', 'test_group'])['value'].sum().rename('denominator_count').reset_index()

    # Об'єднання в один датафрейм через outer join, поєднуємо за двома колонками для правильної агрегації
    joined_data = pd.merge(numerator_counts, denominator_counts, on=['test', 'test_group'], how='outer')

    # Обчислюємо конверсію
    joined_data['conversion'] = joined_data['numerator_count'] / joined_data['denominator_count']

    # Робимо таблиці по групам
    group1 = joined_data[joined_data['test_group'] == 1].rename(columns={'numerator_count': 'numerator_count_1', 'denominator_count': 'denominator_count_1', 'conversion': 'conversion_1'}).drop(columns='test_group')
    group2 = joined_data[joined_data['test_group'] == 2].rename(columns={'numerator_count': 'numerator_count_2', 'denominator_count': 'denominator_count_2', 'conversion': 'conversion_2'}).drop(columns='test_group')

    # об'єднуємо групи по test
    df_with_result = pd.merge(group1, group2, on='test', how='outer')

    # Z-тест
    z_stats = []
    p_values = []
    # df_with_result.iterrows() повертає ітератор, який дає кожен рядок датафрейму.
    # індекс рядка (_, бо він не потрібен),
    # рядок (row)
    # count, nobs стандарт в у Z-тесті
    for _, row in df_with_result.iterrows():
    # кількість успіхів (numerator) у кожній групі
        count = [row['numerator_count_1'], row['numerator_count_2']]
    # кількість спроб (denominator) у кожній групі
        nobs = [row['denominator_count_1'], row['denominator_count_2']]

        stat, pval = proportions_ztest(count, nobs)
        z_stats.append(stat)
        p_values.append(pval)
    # додаємо нові колонки
    df_with_result['difference_%'] = (df_with_result['conversion_2'] - df_with_result['conversion_1']) / df_with_result['conversion_1'] * 100
    df_with_result['z_statistic'] = z_stats
    df_with_result['p_value'] = p_values
    df_with_result['significance'] = df_with_result['p_value'] < alpha

    # Додаємо описові колонки
    df_with_result.insert(1, 'metric', f"{numerator_event} / {denominator_event}")
    df_with_result.insert(2, 'numerator_event', numerator_event)
    df_with_result.insert(3, 'denominator_event', denominator_event)

    return df_with_result


## **Створення функція для роботи з фільтрами:**

In [ ]:
def calculate_ab_metrics_filtered(df: pd.DataFrame, numerator_event: str, denominator_event: str, filter_column: str = None, filter_value: str = None, alpha_f: float = 0.05) -> pd.DataFrame:
    # Фільтрує датасет або працює з повним датасетом, якщо фільтри не задані, викорстовує попередню функцію з тими ж аргументами
    # За замочуванням None для оборбки випадків коли не буде аргументів фільтрації

    # застосовуємо фільтр
    if filter_column is not None and filter_value is not None:
        # Якщо фільтр заданий, фільтруємо дані
        df_with_filter = df[df[filter_column] == filter_value].copy()
    else:
        # Якщо фільтр не заданий, використовуємо повний датасет
        df_with_filter = df.copy()

    # основна функція
    filter_result = calculate_ab_metrics(df=df_with_filter, numerator_event=numerator_event, denominator_event=denominator_event, alpha=alpha_f)

    return filter_result

## **Використання функції з різними метриками:**

### **`add_payment_info / session` (основна функція)**

In [ ]:
add_payment_info_df = calculate_ab_metrics(df=ab_data, numerator_event='add_payment_info', denominator_event='session')

add_payment_info_df

,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,add_payment_info / session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,2,add_payment_info / session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
2,3,add_payment_info / session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,-0.643172,0.520112,False
3,4,add_payment_info / session,add_payment_info,session,3731,105079,0.035507,3601,105141,0.034249,-3.541234,1.571106,0.116158,False


### **`add_shipping_info / session` (основна функція)**

In [ ]:
add_shipping_info_df = calculate_ab_metrics(df=ab_data, numerator_event='add_shipping_info', denominator_event='session')

add_shipping_info_df

,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,add_shipping_info / session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
1,2,add_shipping_info / session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
2,3,add_shipping_info / session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,1.413727,0.157442,False
3,4,add_shipping_info / session,add_shipping_info,session,5128,105079,0.048801,4956,105141,0.047137,-3.411125,1.785795,0.074132,False


### **`begin_checkout / session` (функція з фільтрами але без їх використання)**

In [ ]:
begin_checkout_df = calculate_ab_metrics_filtered(df=ab_data, numerator_event='begin_checkout', denominator_event='session') # filter_column та filter_value пропущені

begin_checkout_df

,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,begin_checkout / session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
1,2,begin_checkout / session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False
2,3,begin_checkout / session,begin_checkout,session,9532,70047,0.136080,9264,70439,0.131518,-3.352445,2.511389,0.012026,True
3,4,begin_checkout / session,begin_checkout,session,12555,105079,0.119482,12267,105141,0.116672,-2.351523,1.995998,0.045934,True


### **`new_accounts / session` (функція з фільтрами але без їх використання)**

In [ ]:
new_accounts_df = calculate_ab_metrics_filtered(df=ab_data, numerator_event='new account', denominator_event='session') # filter_column та filter_value пропущені

new_accounts_df

,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,new account / session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
1,2,new account / session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,False
2,3,new account / session,new account,session,5856,70047,0.083601,5822,70439,0.082653,-1.133880,0.643489,0.519907,False
3,4,new account / session,new account,session,8984,105079,0.085498,8687,105141,0.082622,-3.362896,2.375457,0.017527,True


### **`add_shipping_info / session` з фільтрами по каналу 'Organic Search'**
**(функція з фільтрами, приклад використання)**

In [ ]:
organic_search_add_shipping_info_df = calculate_ab_metrics_filtered(df=ab_data, numerator_event='add_payment_info', denominator_event='session_start', filter_column='channel', filter_value='Organic Search',alpha_f=0.01)

print("A/B тест для сегмента 'channel' = 'Organic Search' (alpha=0.01)")
organic_search_add_shipping_info_df

A/B тест для сегмента 'channel' = 'Organic Search' (alpha=0.01)


,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,add_payment_info / session_start,add_payment_info,session_start,640,15963,0.040093,514,15903,0.032321,-19.384491,3.712987,0.000205,True
1,2,add_payment_info / session_start,add_payment_info,session_start,698,17751,0.039322,597,17768,0.033600,-14.551747,2.876792,0.004017,True
2,3,add_payment_info / session_start,add_payment_info,session_start,1036,25154,0.041186,1098,25584,0.042917,4.203233,-0.971309,0.331394,False
3,4,add_payment_info / session_start,add_payment_info,session_start,1092,38879,0.028087,1063,38616,0.027527,-1.992700,0.473796,0.635646,False


## **Об'єднання основних метрик в один датафрейм**

In [ ]:
total_ab_df = pd.concat([add_payment_info_df, add_shipping_info_df, begin_checkout_df, new_accounts_df], ignore_index=True)
total_ab_df

,test,metric,numerator_event,denominator_event,numerator_count_1,denominator_count_1,conversion_1,numerator_count_2,denominator_count_2,conversion_2,difference_%,z_statistic,p_value,significance
0,1,add_payment_info / session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,2,add_payment_info / session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
2,3,add_payment_info / session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,-0.643172,0.520112,False
3,4,add_payment_info / session,add_payment_info,session,3731,105079,0.035507,3601,105141,0.034249,-3.541234,1.571106,0.116158,False
4,1,add_shipping_info / session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
5,2,add_shipping_info / session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
6,3,add_shipping_info / session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,1.413727,0.157442,False
7,4,add_shipping_info / session,add_shipping_info,session,5128,105079,0.048801,4956,105141,0.047137,-3.411125,1.785795,0.074132,False
8,1,begin_checkout / session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
9,2,begin_checkout / session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False


## **Висновки**

Провівши статистичні дослідження результатів А/В тестування, ми можемо констатувати наявінсть статистичної значущусті змін у контрольній та тестовий групах за наступними метриками та тестами:

**Тест 1:**
1. Конверсія додавання платіжної інформації (add_payment_info) статистично значуще збільшилася на 12,54% у результаті змін, впроваджених у тесті 1.
2. Також зміни, впровадженя у тесті 1 статистично значуще збільшили конверсію додавання інформції про доставку (add_shipping_info) на 6.56%.
3. У результаті змін у тесті 1 конверсія початку оплати (begin_checkout) статистично значуще збільшилася на 6,66%.

*Варто впровадити зміни, які перевірялися у тесті 1 для всіх користувачів.*

**Тест 2** не приніс няіких статистично значимих результітв, впровадження цих змін не є доцільним.

**Тест 3** негативно вплинув на конверсію  початку оплати (begin_checkout), статистично значуще зменшивши її на 3.35%. *Ці зміни не врато впроваджувати, також є доцільним додатково дослідити, з чим саме пов'язаний цей негативний вплив.*

**Тест 4** також мав негативний впливи на такі метрики:
1. Конверсія початку оплати (begin_checkout) статистично значуще зменшилася на 2,35%.
2. Конверсія створення нових аккаунтів (new account) статистично значуще зменшилася на 3,36%.
*Зміни тесту 4 також не варто впроваджувати, потрібно дослідити причини негативного впливу.*  


## **Збереження датафрейму**

In [ ]:
file_path = '/content/drive/MyDrive/Mate/Portfolio/total_ab_df.csv'
total_ab_df.to_csv(file_path, index=False)